In [28]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os
checkpoint_path = "training_ResNet_52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [29]:
d_rate=0 #The rate of droupout for the layers in the ResNet
act='swish'#The activation function use ('relu','swish','elu','gelu','selu')
lr=0.01 #The default learning rate Default 0.01
mom=0.9 #The momentum parameter Default 0.9
test_split=1 #Default 1 and range[1,0)
#Reducing test_split will make the model faster but will also reduce the accuracy of the Validation metric
#On changing test_split run the lower initialisation cell, marked below**

In [2]:
df_data=pd.read_csv("DataSetNormalised.csv")

In [4]:
df_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,0,0.050286,0.369601,0.564642,0.021253,0.302234,0.062910,0.458154,0.186402,0.0,...,0.497833,0.486773,1.0,0.031874,0.038707,0.113156,0.055765,0.038036,0.061924,0.067874
1,1,0.053521,0.567601,0.603004,0.097277,0.410338,0.097729,0.406137,0.043934,1.0,...,0.271923,0.499667,0.0,0.005662,0.031445,0.114120,0.059877,0.041314,0.062247,0.058134
2,2,0.044327,0.802000,0.030564,0.029457,0.622152,0.098682,0.715906,0.896692,0.0,...,0.725909,0.758206,0.0,0.020806,0.045090,0.114118,0.057983,0.042641,0.050959,0.055869
3,3,0.090465,0.320000,0.768391,0.129367,0.753152,0.168172,0.222741,0.229380,0.0,...,0.364120,0.109674,0.0,0.021726,0.041143,0.115758,0.042198,0.046364,0.065316,0.077971
4,4,0.070221,0.566001,0.936645,0.057322,0.154448,0.055491,0.319775,0.235269,0.0,...,0.425140,0.532329,0.0,0.016970,0.057617,0.114235,0.049968,0.061272,0.051523,0.059396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10999995,10999995,0.074864,0.708200,0.531054,0.097118,0.345951,0.224923,0.358620,0.879182,0.0,...,0.480493,0.841457,1.0,0.018622,0.028159,0.116204,0.065760,0.044546,0.050472,0.054919
10999996,10999996,0.029067,0.292001,0.818375,0.061103,0.391357,0.088460,0.558688,0.017989,1.0,...,0.456486,0.800867,1.0,0.018739,0.039215,0.119606,0.075558,0.047622,0.057437,0.066455
10999997,10999997,0.036016,0.659000,0.936008,0.055004,0.560670,0.097729,0.508838,0.507019,1.0,...,0.817271,0.991564,0.0,0.006538,0.032048,0.114361,0.053177,0.103900,0.076656,0.074661
10999998,10999998,0.076397,0.524201,0.133552,0.121080,0.332522,0.087833,0.287096,0.866448,0.0,...,0.780094,0.123205,0.0,0.019038,0.033915,0.141387,0.057126,0.016474,0.041420,0.048286


In [3]:
df_label=pd.read_csv("LabelSet.csv")

In [4]:
x=np.array(df_data.iloc[:,1:])

In [5]:
y=np.array(df_label.iloc[:,1])

In [6]:
x=x.astype(np.float32)

In [27]:
#On changing test_split run this--------**
size=10000000 + test_split*1000000
x_train=x[:][:10000000]
y_train=y[:][:10000000]
x_test=x[:][10000000:size]
y_test=y[:][10000000:size]

In [8]:
y_train.shape

(10000000,)

In [18]:
def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    #There will be 3 layers in total for the sequential connection
    #Activation will be specified by the user as per the Keras Documentation
    #no_nodes denote the size of the model which will be built
    #Applying Batch Normalisation to size the inputs
    #Layer 1 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 2-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 3-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 4-------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('swish')(X)
    #--------------------------------------------------------
    X_shortcut2=X
     #Layer 5 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 6-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 7-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X,X_shortcut2])
    #Layer 8-------------------------------------------------
    X=tf.keras.layers.Activation('swish')(X)
    return X
    

In [19]:
inputs=tf.keras.layers.Input(shape=(28,))                    #Keras input layer
X=tf.keras.layers.Dense(300,activation=act)(inputs)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     #keras output layer
model=tf.keras.Model(inputs,outputs)                         #Model declaration

In [20]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28)]         0                                            
__________________________________________________________________________________________________
dense_35 (Dense)                (None, 300)          8700        input_3[0][0]                    
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 300)          90300       dense_35[0][0]                   
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 300)          1200        dense_36[0][0]                   
_______________________________________________________________________________________

In [23]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=lr,
        beta_1=mom,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [24]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF052190> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002B0EF0DD610>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EF2A1370> and <tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF052190>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF2AD970> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EF2A1370>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EEEE56A0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002B0EEF368E0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF03E250> and <tensorflow.python.keras.layers.merge.Add object at 0x000002B0EF17F700>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EEF48DC0> and <tensorflow.python.keras.layers.core.Activation object at 0x000002B0EF17F9D0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EEF48640> and <tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF188B20>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EED898E0> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EF1219D0>).

Two checkpoint references resolved to different objects (<tensorflow.python.kera


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EF5AE700> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002B0EEFCEE80>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF5D1B50> and <tensorflow.python.keras.layers.core.Dense object at 0x000002B0EEE57250>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000002B0EF69C190> and <tensorflow.python.keras.layers.core.Dense object at 0x000002B0EEE0CCD0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002B0EF631DF0> and <tensorflow.python.keras.layers.core.Dropout object at 0x000002B0EF440F40>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense objec

In [25]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=True,
                                                 save_best_only=True)

model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=10240,
    validation_data=(x_test,y_test),
    callbacks=[cp_callback]
)

Epoch 1/100
 27/977 [..............................] - ETA: 3:52 - loss: 0.6328 - accuracy: 0.6339 - auc: 0.6888

KeyboardInterrupt: 

In [29]:
model.save('ResNet_52')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ResNet_19\assets
